# Python Modules

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime, timedelta
import time

In [ ]:
key = {'api_key': "INSERT API KEY"}

# Daily Data Pull Functions

In [ ]:
def get_daily_pull(data_df, key):
    """
    Pulls guild info from guild data endpoint 20 guilds at a time
    input: blank dataframe
    output: dataframe with all guild info
    """
    flag = True
    page_num = 1
    while flag:
        # API rate limits to 40 calls per min
        if page_num == 30 or page_num == 60 or page_num == 90:
            time.sleep(60)

        all_guild_url = f"https://api.simple-mmo.com/v1/guilds/all?page={page_num}"
        all_guild_response = requests.post(all_guild_url, data = key)
        if all_guild_response.ok:
            #print(f"Success! Status Code = {all_guild_response.status_code}")
            # load API content into json
            all_guild_response_dict = json.loads(all_guild_response.text)
            # stores the list of guilds and their data 
            data = all_guild_response_dict['data']

            # API returns a blank page if for loop has iterated through all the guilds instead of a different status code than 200
            # this catches the blank data and ends the for loop
            if data == []:
                flag = False
            else:
                # append list of dictionaries into a dataframe
                data_df = data_df.append(data, ignore_index = True, sort = False)
                page_num += 1
        else:
            print("Failure!")
            print(all_guild_response.status_code)
            flag = False
    
    return data_df

In [ ]:
def remove_inactive_guilds(guild_df):
    """
    Removes all guilds where they haven't earn any exp since the guild update (v9.0)
    input: dataframe of all guild info
    output: dataframe of "active" guilds
    """
    active_guilds_df = guild_df[guild_df['exp'] != 0]
    return active_guilds_df

In [ ]:
def sort_guilds(cleaned_guild_df):
    """
    Sorts active guilds by ascending order
    input: active guilds dataframe
    output: sorted active guilds dataframe
    """
    sorted_df = cleaned_guild_df.sort_values(by = ['exp'], ascending = False)
    return sorted_df

In [ ]:
def create_standing(sorted_guild_df):
    """
    Creates a standing column in dataframe. This is the rank like a leaderboard
    input: sorted, active guild dataframe
    output: sorted, active guild dataframe
    """
    sorted_guild_df['standing'] = sorted_guild_df['exp'].rank(ascending = False, method = 'first')
    return sorted_guild_df

In [ ]:
def get_guild_ids(daily_df):
    """
    Get list of guild ids from daily guild data
    Input: DataFrame
    Output: List
    """
    guild_ids = daily_df['id'].tolist()
    
    return guild_ids

In [ ]:
def post_request(guild_id):
    """
    Conduct a POST request to the SMMO Guild Members End Point
    Input: string
    Output: requests.Response object
    """
    url = f"https://api.simple-mmo.com/v1/guilds/members/{guild_id}"
    mem_response = requests.post(url, data = key)
    
    return(mem_response)

In [ ]:
def get_guild_mems(guild_list):
    """
    For each guild, get the number of guild members currently in the guild.
    Input: list 
    Output: DataFrame
    """
    counter = 0
    ans_df = pd.DataFrame()

    for gid in guild_list:
        # API endpoint has a rate limit of 40 calls per minute
        # to stay safe add minute of delay after 30 calls to end point
        if counter % 30 != 0:
            member_response = post_request(gid)
            counter += 1
            print(counter)
        else:
            time.sleep(60)
            member_response = post_request(gid)
            counter += 1
            print(counter)
        # take text data, make it a dictionary then build a temp dataframe which will be concated with end dataframe of all data
        if member_response.ok:
            glen_dict = {}
            mem_dict = json.loads(member_response.text)
            glen_dict[gid] = len(mem_dict)
            print(glen_dict)
            temp_df = pd.DataFrame({'Guild': list(glen_dict.keys()), 'Number of Members': list(glen_dict.values())})
            ans_df = pd.concat([ans_df, temp_df], ignore_index = True)
        
        else:
            print("Failure!")
            print(mem_reponse.status_code)
    
    return ans_df

In [ ]:
def create_mem_count(guild_mem_count_df, overall_df):
    """
    Take DataFrame of guild member counts and combine it with the overall daily guild DataFrame.
    Input: DataFrame
    Output: DataFrame
    """
    
    member_count_list = guild_mem_count_df['Number of Members'].tolist()
    overall_df['Member Count'] = member_count_list
    
    return overall_df

In [ ]:
def create_date(ranked_df, date):
    """
    Creates a date column in dataframe
    input: ranked dataframe
    input: date object (YYYY-MM-DD)
    output: dataframe
    """
    ranked_df['date'] = date
    return ranked_df

In [ ]:
def add_to_excel(complete_df, full_date):
    """
    Writes finalized dataframe to a specific worksheet in the excel workbook
    input: dataframe
    input: date object (Week Day - YYYY-MM-DD)
    """
    with pd.ExcelWriter(r"INSERT FILE PATH", engine = "openpyxl",
                        mode = "a") as writer:
        complete_df.to_excel(writer, sheet_name = full_date, index = False)

# Main Code

In [ ]:
data_df = pd.DataFrame()
# daily pull of guilds
starter_guild_df = get_daily_pull(data_df, key)

In [ ]:
# removing "inactive" guilds
active_df = remove_inactive_guilds(starter_guild_df)

In [ ]:
# sorting guilds by ascending order 
sorted_active_guilds = sort_guilds(active_df)

In [ ]:
# create standing rank of guilds
leaderboard_guilds = create_standing(sorted_active_guilds)

In [ ]:
# getting guild ids from daily pull
guild_ids_list = get_guild_ids(leaderboard_guilds)

In [ ]:
# getting membership numbers of each guild
guild_mem_df = get_guild_mems(guild_ids_list)

In [ ]:
# add member count to final DataFrame
total_df = create_mem_count(guild_mem_df, leaderboard_guilds)

In [ ]:
date = date.today()
final_df = create_date(total_df, date)

In [ ]:
day_of_week = datetime.today().strftime("%A")
full_date = f"{day_of_week} - {date}"
add_to_excel(final_df, full_date)